In [95]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import math

In [96]:
data_amundi = pd.read_csv('./data/amundi-msci-wrld-ae-c.csv')
data_amundi.name = 'amundi-msci-wrld-ae-c'

data_db = pd.read_csv('./data/db-x-trackers-ii-global-sovereign-5.csv')
data_db.name = 'db-x-trackers-ii-global-sovereign-5'

data_ishares = pd.read_csv('./data/ishares-global-corporate-bond-$.csv')
data_ishares.name = 'ishares-global-corporate-bond-$'

data_spdr = pd.read_csv('./data/spdr-gold-trust.csv')
data_spdr.name = 'spdr-gold-trust'

data_us = pd.read_csv('./data/usdollar.csv')
data_us.name = 'usdollar'

data = [data_amundi, data_db, data_ishares, data_spdr, data_us]

In [97]:
for dataframe in data:

    for index, row in dataframe.iterrows():
        dataframe.loc[index, 'Change %'] = dataframe.loc[index, 'Change %'][0:4]
        dataframe.loc[index,'Change %'] = float(dataframe.loc[index,'Change %'])


In [98]:
def returns(assets):
    current_value = assets["Price"].tail(1)
    buy_amount = assets.loc[0, 'Price']
    return (current_value - buy_amount)/buy_amount *100

for assets in data:
    print(assets.name ,'\n Returns:', float(returns(assets)))

amundi-msci-wrld-ae-c 
 Returns: -4.88317528595171
db-x-trackers-ii-global-sovereign-5 
 Returns: -0.22061140876142543
ishares-global-corporate-bond-$ 
 Returns: -3.7707764822624736
spdr-gold-trust 
 Returns: -19.29244225162594
usdollar 
 Returns: 7.171447631754506


In [99]:
def volatility(assets):
    N = len(assets)
    muX = assets['Change %'].sum()/len(assets)
    sigmaX = math.sqrt(np.square(np.subtract(assets['Change %'], muX).mean()))
    return sigmaX/muX*100

for assets in data:
    print(assets.name ,'\n Volatility:', float(volatility(assets)))

amundi-msci-wrld-ae-c 
 Volatility: 9.969930945960345e-15
db-x-trackers-ii-global-sovereign-5 
 Volatility: 4.323298382496717e-14
ishares-global-corporate-bond-$ 
 Volatility: 4.1315551488913264e-14
spdr-gold-trust 
 Volatility: 1.134037818820385e-14
usdollar 
 Volatility: -3.9254559448960484e-13


In [100]:
def value(shares, assets):
    return sum(shares * assets['Price'])

## Handle missing data

In [101]:
# find missing dates for every asset

#idx = pd.date_range('2020-01-01', '2020-12-31')

# data_amundi.set_index('Date')
# data_ishares.set_index('Date')
# data_us.set_index('Date')
# data_db.set_index('Date')
# data_spdr.set_index('Date')

data_amundi['Date'] =  pd.to_datetime(data_amundi['Date'], format='%Y-%m-%d')
data_amundi = data_amundi.sort_values(by=['Date'], ascending=[True])
data_amundi.set_index('Date', inplace=True)

data_amundi_fill = data_amundi.resample('D').ffill().reset_index()
print(data_amundi_fill)



          Date   Price Change %
0   2020-01-02  194.59     0.97
1   2020-01-03  194.12     -0.2
2   2020-01-04  194.12     -0.2
3   2020-01-05  194.12     -0.2
4   2020-01-06  193.81     -0.1
..         ...     ...      ...
360 2020-12-27  203.19     0.32
361 2020-12-28  204.02     0.41
362 2020-12-29  204.00     -0.0
363 2020-12-30  203.32     -0.3
364 2020-12-31  204.58     0.62

[365 rows x 3 columns]


# Processing of all 126 asset allocation instances
- open portfolios/portfolio_allocations.csv
- calculate number of shares for each allocation depending on price at the relevant dates
-

In [102]:
import csv

with open('portfolios/portfolio_allocations.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    header = []
    header = next(reader)
    print(f"Header of csv is {header}, describes the order of allocation of assets in the portfolio")

    rows = []
    for row in reader:
            rows.append(row)


Header of csv is ['ST', 'CB', 'PB', 'GO', 'CA'], describes the order of allocation of assets in the portfolio


For every row we have the distribution of assets
- extract the price of each asset on first and last day of interest to calculate yearly return and volatility
- for reference and ease of comprehension

| Reference |   Description   |   Variable   |
|:-----------|:---------:|:---------------:|:------------:|
| ST        |     stocks      | data_amundi  |
| CB        | corporate bonds | data_ishares |
| PB        |  public bonds   |   data_db    |
| GO        |      gold       |  data_spdr   |
| CA        |      cash       |   data_us    |

- Regarding the use of share prices, in the case of cash (CA), the price is not obtained from the
US Dollar index (described first part of assignment), but rather the purchase price (or cost) is
always considered as 1$.

In [103]:
START_DATE = '01/01/2020'
STOP_DATE = '12/31/2020'

#order the dataframes by date
data_amundi = data_amundi.sort_values(by=["Date"], ascending=True)
data_db = data_db.sort_values(by=["Date"], ascending=True)
data_ishares = data_ishares.sort_values(by=["Date"], ascending=True)
data_spdr = data_spdr.sort_values(by=["Date"], ascending=True)
data_us = data_us.sort_values(by=["Date"], ascending=True)

#values of interest are the following
start_ST = data_amundi['Price'].iloc[0]
start_CB = data_ishares['Price'].iloc[0]
start_PB = data_db['Price'].iloc[0]
start_GO = data_spdr['Price'].iloc[0]
start_CA = 1.0

end_ST = data_amundi['Price'].iloc[-1]
end_CB = data_ishares['Price'].iloc[-1]
end_PB = data_db['Price'].iloc[-1]
end_GO = data_spdr['Price'].iloc[-1]

#determine if dollar has gained or loss value from start to end of year,
#necessary to calculate the return on the cash asset investment
end_CA = data_us['Price'].iloc[-1]/data_us['Price'].iloc[0]

#loop through portfolio instances
for el in rows:
    #number of shares acquired on 01/01/2020
    sh_ST = float(el[0])/start_ST
    sh_CB = float(el[1])/start_CB
    sh_PB = float(el[2])/start_PB
    sh_GO = float(el[3])/start_GO
    sh_CA = float(el[4])/start_CA

    sh_arr = [sh_ST, sh_CB, sh_PB, sh_GO, sh_CA]

    #calculate buy_amount
    buy_amount = 0
    buy_amount += (sh_ST * start_ST) + \
                  (sh_CB * start_CB) + \
                  (sh_PB * start_PB) + \
                  (sh_GO * start_GO) + \
                  (sh_CA * start_CA)

    #calculate current_value
    current_value = 0
    current_value += (sh_ST * end_ST) + \
                     (sh_CB * end_CB) + \
                     (sh_PB * end_PB) + \
                     (sh_GO * end_GO) + \
                     (sh_CA * end_CA)

    return_portfolio = ((current_value - buy_amount)/buy_amount)*100

    #calculate volatility

    #first calculate the sequence of Values = {value_i}. The volatility will be calculated in function
    #of this sequence
    values = pd.DataFrame()
    values['Date'] = []
    values['val'] = []
    #initial price and #shares is in
    #value_i = sum(#shares bought on 01/01/2020  * buy_price) over 1 year

    #loop through 365 items (fill missing spots)


    for id, asset in enumerate(data):
        #num_shares of asset now in consideration
        #for every day of the period of interest
        num_shares = sh_arr[id]
        #retrieve price on i_th day

        #append return to rows


## Calculate volatility of portfolio